In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [5]:
df=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

In [122]:
mandat_dep=df.sort_values(by='Département')


In [ ]:
mandat_dep['âge député']=mandat_dep[df_depu['date naissance'][df_depu['nom'==mandat_dep[mandat_dep['nom']]]]-mandat_dep['date']]

In [123]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements mentionnés
dep_possible.remove(" Département d'élection inconnu")
for dep in dep_possible :#on enlève les nan
    if type(dep)!=str :
        dep_possible.remove(dep)
#dep_possible#attention, il y a souvent un espace avant !


In [124]:
mandat_dep.count()#on vérifie qu'il n'y a pas trop de manques

Unnamed: 0          37761
Département         36431
Groupe              33607
Législature         37748
Mandat              37748
Régime politique    37745
nom du député       37761
dtype: int64

In [125]:
mandat_dep.set_index('Département', inplace=True)#on met département en index pour pouvoir scinder par départements

In [126]:
liste_dep={}#on crée un dictionnaire où on va mettre les dataframes par départements
for dep in dep_possible:#pour chaque département
    mand_du_dep=mandat_dep.loc[dep]#on récupère le dataframe des mandats dans le département     
    liste_dep[dep]=mand_du_dep#et on l'ajoute au dictionnaire


In [127]:
carac_dep={}
for dep in dep_possible :
    nbre_mandat=liste_dep[dep]['nom du député'].count()#on compte le nombre total de mandat dans le département
    cpte_depute=0#on crée la variable qui va compter le nombre de députés différents
    compte=[]#on crée une liste où on va mettre les députés déjà comptés
    for depute in liste_dep[dep]['nom du député'] :#on boucle sur les noms de députés
        if depute not in compte :#si on ne l'a pas déjà compté
            cpte_depute+=1#on rajoute un au compte
            compte.append(depute)#et on indique qu'on l'a désormais compté
    carac_dep[dep]=pd.DataFrame(['nombre_mandat', 'nombre député','nombre moyen de mandats par député'], [nbre_mandat, cpte_depute, nbre_mandat/cpte_depute])#on crée un dataframe avec les informations
    

